# Autonomous Claims Adjudication using Agent Workflows with Strands Agents

## Understanding Multi-Agent Systems and Swarm Intelligence

An agent swarm is a collection of autonomous AI agents working together to solve complex problems through collaboration. Inspired by natural systems like ant colonies or bird flocks, agent swarms leverage collective intelligence where the combined output exceeds what any single agent could produce. By distributing tasks and sharing information, swarms can tackle complex problems more efficiently and effectively than individual agents working in isolation.

Multi-agent systems consist of multiple interacting intelligent agents within an environment. These systems enable:

- *Distributed Problem Solving*: Breaking complex tasks into subtasks for parallel processing
- *Information Sharing*: Agents exchange insights to build collective knowledge
- *Specialization*: Different agents focus on specific aspects of a problem
- *Redundancy*: Multiple agents working on similar tasks improve reliability
- *Emergent Intelligence*: The system exhibits capabilities beyond those of its individual components

Swarm intelligence emphasizes:

1. *Decentralized Control*: No single agent directs the entire system
2. *Local Interactions*: Agents primarily interact with nearby agents
3. *Simple Rules*: Individual agents follow relatively simple behaviors
4. *Emergent Complexity*: Complex system behavior emerges from simple agent interactions

### Autonomous Claims Adjudication

Claims adjudication is the process by which insurance companies process, review, validate, and assess claims to determine if they should be paid, adjusted, or denied.

Here is a Generic Example of a Claim Adjudication Workflow for Auto-insurnace

NOTE: THIS IS JUST AN EXAMPLE FOR ILLUSTRATION PURPOSES. 




<p align="center">
    <img src="./images/ClaimsAdjudicationFlow.png">
</p>

### Workflow Pattern - Sequential

In the Claims Adjudication Flow, there are tasks that have dependencies and   need to be completed before moving to the next task.
We can use a Swarm Sequential pattern where each agent completes its task before passing the result to the next agent in the chain. 


#### Agentic Flow:
An agent that receives an insurance claim (First Notification of Loss), retrieves policy details, validates information against external sources (e.g., repair shop estimates), completes appraisal or damage assessments, and approves payment or flags the claim for human review.

Pattern —> Workflow with Sequential process


<p align="center">
    <img src="./images/ClaimProcessing.png">
</p>

## 1. Quick Start with Swarm tool

The Strands Agents SDK provides a built-in swarm tool that simplifies the implementation of multi-agent systems, offering a quick start for users. This tool implements the shared memory.

In [ ]:
!pip install -r requirements.txt
!pip install PyPDF2

In [1]:
from strands import Agent
from strands_tools import swarm
import time
import PyPDF2
import json
import boto3
import yaml
import os
import logging
import base64



In [2]:
def read_file(file_path):
    """
    Read and extract text from a PDF file.
    
    Args:
        file_path (str): Path to the PDF file
        
    Returns:
        str: Extracted text from the PDF
    """
    try:
        # Open the PDF file in binary read mode
        # with open(file_path, 'rb') as file:
        #     # Create a PDF reader object
        #     pdf_reader = PyPDF2.PdfReader(file)
        with open(file_path, 'r') as file:
            file = file.read()
            source_json = json.loads(file)
            
            # # Get number of pages
            # num_pages = len(pdf_reader.pages)
            # print(f"Total pages: {num_pages}")
            
            # # Extract text from each page
            # text = ""
            # for page_num in range(num_pages):
            #     page = pdf_reader.pages[page_num]
            #     text += page.extract_text()
                
            return source_json
    
    except FileNotFoundError:
        return "Error: The file was not found."
    except PyPDF2.errors.PdfReadError:
        return "Error: Invalid PDF file or the file is encrypted."
    except Exception as e:
        return f"Error: {str(e)}"



In [3]:
# Example usage
if __name__ == "__main__":
    # file_path = "data/FNOL.json"  # Replace with your  file path
    file_path = "/home/sagemaker-user/Multi-Agent-Collaboration/WorkFlow_ClaimsAdjudication/data/FNOL.json"
    fnol_event = read_file(file_path)
    # print("fnol_report text:")
    # print(fnol_event)

### Using the Workflow Tool

In [4]:
from strands import Agent
from strands_tools import workflow

# Create an agent with workflow capability
agent = Agent(tools=[workflow])

# Create a multi-agent workflow
agent.tool.workflow(
    action="create",
    workflow_id="comprehensive_claim_adjudication",
tasks=[
    {
        "task_id": "fnol_processing",
        "description": "Extract, validate, and structure FNOL data with completeness checks",
        "system_prompt": f"""You are a Claims Data Extraction Specialist.

            ROLE: Process First Notice of Loss (FNOL) forms and extract structured claim data.
            
            INPUTS: Raw FNOL document/form data {fnol_event}
            OUTPUTS: Structured JSON with validated claim information
            
            INSTRUCTIONS:
            1. Extract key data points:
               - Policy number, claim date, incident location, claimant details
               - Incident description, damages reported, witnesses
               - Supporting documentation references
            
            2. Validate data completeness:
               - Flag missing required fields
               - Identify inconsistencies in dates/locations
               - Check format compliance (policy numbers, contact info)
            
            3. Structure output as JSON with confidence scores for each field
            4. Flag potential data quality issues with specific error codes
            5. If critical information is missing, generate specific follow-up questions
            
            ERROR HANDLING: If form is illegible or severely incomplete, flag for manual review with detailed reasoning.
            
            QUALITY CHECK: Ensure all extracted monetary amounts, dates, and identifiers are properly formatted.""",
        "priority": 5
    },
    
    {
        "task_id": "policy_verification",
        "description": "Verify policy coverage, status, and applicable terms",
        "dependencies": ["fnol_processing"],
        "system_prompt": f"""You are a Policy Verification Specialist.

            ROLE: Validate policy coverage and determine claim eligibility.
            
            INPUTS: Structured claim data, policy database access
            OUTPUTS: Policy verification report with coverage determination
            
            INSTRUCTIONS:
            1. Verify policy status:
               - Active coverage on incident date
               - Premium payment status
               - Policy limits and deductibles
            
            2. Analyze coverage applicability:
               - Match incident type to covered perils
               - Check for exclusions or limitations
               - Identify applicable sub-limits
            
            3. Calculate coverage amounts:
               - Maximum payout potential
               - Deductible applications
               - Co-insurance factors
            
            4. Flag policy issues:
               - Lapsed coverage, exclusions, fraud indicators
               - Material misrepresentation concerns
            
            OUTPUT FORMAT: Structured report with coverage determination, limits, and any red flags.
            
            DECISION LOGIC: If policy is invalid or claim not covered, provide clear denial reasoning with policy section references.""",
        "priority": 4
    },

    {
        "task_id": "fraud_detection",
        "description": "Analyze claim for fraud indicators and risk assessment",
        "dependencies": ["fnol_processing"],
        "system_prompt": f"""You are a Fraud Detection Specialist.

            ROLE: Identify potential fraud indicators and assess claim authenticity.
            
            INPUTS: Claim data, historical patterns, external data sources
            OUTPUTS: Fraud risk assessment with detailed analysis
            
            FRAUD INDICATORS TO ANALYZE:
            1. Timeline inconsistencies (reporting delays, convenient timing)
            2. Claim patterns (frequency, amounts, incident types)
            3. Documentation quality (photos, estimates, witness statements)
            4. Claimant behavior (evasiveness, excessive pressure, coaching signs)
            5. Damage consistency (impact patterns, wear indicators)
            
            RISK SCORING:
            - LOW (0-3): Standard processing
            - MEDIUM (4-6): Enhanced investigation required
            - HIGH (7-10): Special Investigation Unit referral
            
            ANALYSIS REQUIREMENTS:
            - Cross-reference similar claims in database
            - Verify incident location and timing plausibility
            - Assess damage patterns against reported cause
            - Check for staging indicators
            
            OUTPUT: Risk score with specific indicators found and recommended actions.""",
        "priority": 4
    },

    {
        "task_id": "damage_appraisal",
        "description": "Comprehensive damage assessment and repair cost estimation",
        "dependencies": ["policy_verification", "fraud_detection"],
        "system_prompt": f"""You are a Damage Appraisal Specialist.

            ROLE: Conduct thorough damage assessment and provide accurate repair cost estimates.
            
            INPUTS: Claim details, photos, estimates, policy coverage info
            OUTPUTS: Detailed appraisal report with itemized repair costs
            
            APPRAISAL PROCESS:
            1. Damage Analysis:
               - Categorize damage by type and severity
               - Distinguish pre-existing vs. incident-related damage
               - Assess structural vs. cosmetic damage
            
            2. Cost Estimation:
               - Use current market rates for materials/labor
               - Consider regional cost variations
               - Factor in quality/grade matching requirements
               - Include necessary vs. improvement upgrades
            
            3. Validation Steps:
               - Compare submitted estimates with market standards
               - Verify contractor credentials and estimates
               - Check for inflated or unnecessary repairs
            
            4. Documentation:
               - Detailed photo analysis with annotations
               - Line-item cost breakdown
               - Justification for any estimate adjustments
            
            QUALITY STANDARDS:
            - All estimates within 10% industry standard ranges
            - Clear separation of covered vs. non-covered repairs
            - Depreciation calculations where applicable
            
            RED FLAGS: Identify overpriced estimates, unnecessary repairs, or non-incident damage claims.""",
        "priority": 3
    },

    {
        "task_id": "settlement_calculation",
        "description": "Calculate final settlement amount with all applicable adjustments",
        "dependencies": ["damage_appraisal"],
        "system_prompt": f"""You are a Settlement Calculation Specialist.

            ROLE: Determine final claim payout with precise calculations and proper adjustments.
            
            INPUTS: Appraisal report, policy terms, coverage verification
            OUTPUTS: Final settlement calculation with detailed breakdown
            
            CALCULATION COMPONENTS:
            1. Base Settlement:
               - Approved repair costs
               - Replacement cost vs. actual cash value
               - Applicable coverage limits
            
            2. Adjustments:
               - Deductible applications
               - Depreciation calculations
               - Betterment considerations
               - Sales tax implications
            
            3. Additional Considerations:
               - Loss of use/additional living expenses
               - Code upgrade requirements
               - Salvage value deductions
            
            VERIFICATION REQUIREMENTS:
            - Double-check all mathematical calculations
            - Ensure compliance with policy terms
            - Validate against coverage limits
            - Confirm proper deductible application
            
            OUTPUT FORMAT:
            - Settlement summary with line-item breakdown
            - Payment authorization details
            - Required documentation for disbursement
            - Any conditions or requirements for payment
            
            COMPLIANCE: Ensure all calculations comply with state regulations and company guidelines.""",
        "priority": 2
    },

    {
        "task_id": "final_review",
        "description": "Comprehensive claim review and authorization",
        "dependencies": ["settlement_calculation"],
        "system_prompt": f"""You are a Final Review Specialist.

            ROLE: Conduct final quality assurance and authorize claim settlement.
            
            INPUTS: Complete claim file with all specialist reports
            OUTPUTS: Final authorization decision with comprehensive summary
            
            REVIEW CHECKLIST:
            1. Completeness Verification:
               - All required documentation present
               - Specialist reports consistent and logical
               - No outstanding information gaps
            
            2. Accuracy Validation:
               - Settlement calculations verified
               - Policy applications correct
               - Fraud assessment reasonable
            
            3. Compliance Check:
               - Regulatory requirements met
               - Company guidelines followed
               - Proper authorization levels
            
            4. Risk Assessment:
               - Overall claim validity confirmed
               - Appropriate settlement amount
               - No unresolved red flags
            
            DECISION OUTCOMES:
            - APPROVE: Issue payment with conditions specified
            - DENY: Provide detailed denial reasoning with policy citations
            - INVESTIGATE: Refer for additional investigation with specific requirements
            
            DOCUMENTATION: Create comprehensive claim summary for audit trail and future reference.
            
            ESCALATION: Flag any claims exceeding authority limits or requiring special handling.""",
        "priority": 1
    }
]
)

# # Execute workflow (parallel processing where possible)
# adjudication_report = agent.tool.workflow(action="start", 
#                     workflow_id="comprehensive_claim_adjudication",
#                     )

# # Check results
# # status = agent.tool.workflow(action="status", workflow_id="comprehensive_claim_adjudication")

{'toolUseId': 'tooluse_workflow_967655314',
 'status': 'success',
 'content': [{'text': 'Created workflow comprehensive_claim_adjudication with 6 tasks'}]}

In [ ]:
# Execute workflow (parallel processing where possible)
adjudication_report = agent.tool.workflow(action="start", 
                    workflow_id="comprehensive_claim_adjudication",
                    )

# Check results
status = agent.tool.workflow(action="status", workflow_id="comprehensive_claim_adjudication")

# FNOL Data Extraction and Validation Report

```json
{
  "claimSummary": {
    "claimNumber": "CL-2023-1156789",
    "reportDate": "2023-11-21",
    "policyNumber": "AUTO-P987654321",
    "policyType": "Auto Insurance",
    "incidentDate": "2023-11-19",
    "incidentTime": "17:45:00",
    "completenessScore": 0.98,
    "validationStatus": "VALIDATED"
  },
  "policyDetails": {
    "insurer": "Acme Insurance Company",
    "effectiveDate": "2023-01-15",
    "expirationDate": "2024-01-15",
    "coverageTypes": [
      {"type": "Liability", "limit": "$300,000/$500,000", "confidence": 1.0},
      {"type": "Collision", "deductible": "$500", "confidence": 1.0},
      {"type": "Comprehensive", "deductible": "$250", "confidence": 1.0},
      {"type": "Medical Payments", "limit": "$10,000", "confidence": 1.0},
      {"type": "Uninsured/Underinsured Motorist", "limit": "$250,000/$500,000", "confidence": 1.0},
      {"type": "Rental Reimbursement", "limit": "$40/day, $1,200 maximum", "confidence":

In [ ]:
display(adjudication_report)

### Simple Example

In [ ]:
from strands import Agent
from strands_tools import workflow

# Create an agent with workflow capability
agent = Agent(tools=[workflow])

# Create a multi-agent workflow
agent.tool.workflow(
    action="create",
    workflow_id="Claim_adjudication",
    tasks=[
        {
            "task_id": "fnol_processing",
            "description": "Proccess FNOL forms Extract key data to process Claim",
            "system_prompt": f"""Process {fnol_event}You extract and structure Claim data from FNOL form.""",
            "priority": 5
        },
        {
            "task_id": "appraisal",
            "description": "Analyze Claim data, damages and assess estimated repair cost",
            "dependencies": ["fnol_processing"],
            "system_prompt": "You identify damages and assess estimated repair cost based on Claim data.",
            "priority": 3
        },
        {
            "task_id": "settlement",
            "description": "Generate Claim payout",
            "dependencies": ["appraisal"],
            "system_prompt": "You create a Claim payout report.",
            "priority": 2
        }
    ]
)

# Execute workflow (parallel processing where possible)
agent.tool.workflow(action="start", 
                    workflow_id="Claim_adjudication",
                    )

# Check results
status = agent.tool.workflow(action="status", workflow_id="Claim_adjudication")

In [ ]:
from strands import Agent

# Create specialized agents
fnol_processing = Agent(system_prompt=f"""You extract and structure Claim data from FNOL form.""", 
                        callback_handler=None)
appraisal = Agent(system_prompt=f""""You identify damages and assess estimated repair cost based on Claim data.""",
                                 callback_handler=None)
settlement = Agent(system_prompt=f"""You create a Claim payout report.""")

# Sequential workflow processing
def process_workflow(fnol_report):
    # Step 1: FNOL Processing
    fnol_processing_results = fnol_processing(f"process {fnol_event}")

    # Step 2: Damage Assessment
    appraisal_results = appraisal(f"Analyze these research findings: {fnol_processing_results}")

    # Step 3: Payout Generation
    settlement_results = settlement(f"Create a report based on this analysis: {appraisal_results}")

    return settlement_results

In [ ]:
process_claim = process_workflow(fnol_event)